<h1>Evaluation<h1>

<h5>분류의 성능 평가 지표</h5>

- 정확표(Accuracy)
- 오차행렬(Confusion Matrix)
- 정밀도(Precision)
- 재현율(Recall)
- F1 스코어
- ROC AUC

<br><br>

<h3>4.1 정확도(Accuracy)</h3>
<br><br>

$$
정확도(Accuracy) = \frac{예측 결과가 동일한 데이터 건수}{\text{전체 예측 데이터 건수}}
$$

<br><br>

- 직관적으로 모델 예측 성능 평가 지표<br>
    -> 실제 데이터에서 예측 데이터가 얼마나 같은지 판단하는 지표<br>
    
- 구성에 따라 ML 모델의 성능을 왜곡할 수 있으므로 정확도만 가지고 성능을 평가하진 않음


[예제]
- 사이킷런의 BaseEstimator 클래스를 상속 후 학습 진행 x
- Sex에 따라 생존자를 예측하는 단순한 Classifier 생성

** 사이킷런의 BaseEstimator 상속
-> Customized형태의 Estimator를 개발자가 생성할 수 있음

In [5]:
from sklearn.base import BaseEstimator
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

class MyDummyClassifier(BaseEstimator):
    # fit() 메서드는 아무것도 학습하지 않음
    def fit(self, x, y=None):
        pass
    
    # predict() 메서드는 단순히 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측함
    def predict(self, x):
        pred = np.zeros( (x.shape[0], 1) )
        for i in range (x.shape[0]):
            if x['Sex'].iloc[i]==1:
                pred[i] = 0
            else:
                pred[i] = 1
                
        return pred.ravel()
    

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할
titanic_df = pd.read_csv('data_handling/titanic_train.csv')

titanic_df['Age'] = titanic_df['Age'].fillna(titanic_df['Age'].mean())
titanic_df['Cabin'] = titanic_df['Cabin'].fillna('N')
titanic_df['Embarked'] = titanic_df['Embarked'].fillna('N')

titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]
titanic_df.groupby(['Sex', 'Survived'])['Survived'].count()

# Null 처리 함수
def fillna(df):
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Cabin'] = df['Cabin'].fillna('N')
    df['Embarked'] = df['Embarked'].fillna('N')
    df['Fare'] = df['Fare'].fillna('N')
    # df['Age'].fillna(df['Age'].mean(), inplace=True)
    # df['Cabin'].fillna('N', inplace=True)
    # df['Embarked'].fillna('N', inplace=True)
    # df['Fare'].fillna('N', inplace=True)
    return df

# 머신러닝 알고즘 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df


y_titanic_df = titanic_df['Survived']
x_titanic_df = titanic_df.drop('Survived', axis=1)
x_titanic_df = transform_features(x_titanic_df)
x_train, x_test, y_train, y_test = train_test_split(x_titanic_df, y_titanic_df, test_size=0.2, random_state=0)

myclf = MyDummyClassifier()
myclf.fit(x_train, y_train)

mypredictions = myclf.predict(x_test)
print('Dummy Clssifier의 정확도는: {0:.4f}'.format(accuracy_score(y_test, mypredictions)))


Dummy Clssifier의 정확도는: 0.7877


** 정확도
- 불균형한(imbalanced) 레이블 값 분포에서 ML 모델의 성능을 판단할 경우, 적합한 평가 지표가 아님

** MNIST 데이터 세트 변환
 - 0 ~9 의 픽셀 장보
 - 숫자 Digit 예측에 사용
 - load_digits() 

In [1]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self, x, y):
        pass
    
    # 입력값으로 들어오는 x 데이터 세트의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, x):
        return np.zeros( (len(x), 1), dtype=bool )
    
# 사이킷런의 내장 데이터 세트인 load_digits()를 이용해 MNIST 데이터 로딩
digits = load_digits()

# digits 번호가 7번이면 True이고 이를 astype(int)로 1로 변환, 7번이 아니면 Falsedlrh 0dmfh qusghks
y = (digits.target == 7).astype(int)
x_train, x_test, y_train, y_test = train_test_split( digits.data, y, random_state=11 )

# 불균형 데이터 생성 = y_test
# 불균형한 레이블 데이터 분포도 확인
print('레이블 테스트 세트 크기 :', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

# MyFakeClassifier예측 진행
# Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(x_train, y_train)
fakepred = fakeclf.predict(x_test)
print('모든 예측을 0으로 하여도 정확도는:{:.3f}'.format(accuracy_score(y_test, fakepred)))


레이블 테스트 세트 크기 : (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
Name: count, dtype: int64
모든 예측을 0으로 하여도 정확도는:0.900


predict() 결과값
- np.zeros()를 사용해 모두 0값으로 반환되나
- 테스트 데이터 세트 예측 정화도 = 90%

** 불균형 데이터를 성능 수치로 사용해서는 안됨을 보여줌

** ML 모델 성능 평가 => 4분면의 오차 행렬(Confusion Matrix)
- True/False
- Positive/Negative

<h3>4.2 오차 행렬</h3>

- 오차행렬(Confusion Matrix), 혼동행렬)
- 학습된 분류 모델이 예측을 수행하면서 얼마나 confused한지도 함께 보여주는 지표
- 이진 분류의 예측 오류가 얼마인지
- 어떤 유형의 예측 오류가 발생하고 있는지 함께 나타내는 지표

** 오차행렬 구성
- 4분면의 행렬에서 실제 레이불 클래스 값과 예측 레이블 클래스 값이 어떠한 유형을 가지고 매핑되는지 나타냄
- 예측 클래스와 실제 클래스 값 유형에 따라 TN, FP, FN, TP 형태로 4분면을 채움
- 완쪽, 오른쪽 : 예측된 클래스값 기준
- 위, 아래 : 실제 클래스값 기준

** 형태

                        예측 클래스
                    (Predict Class)

                    Negative(0)  |  Positive(1)
    Negative(0)                  |
                        TN       |          FP
                (True Negative)  |   (False Positive)
    실제클래스  ------------------------------------------
    (Actual Class)               |
                        FN       |          TP     
                (False Negative) |   (True Positive)








TN, FP, FN, TP : 예측 클래스 & 실제 클래스의 Postive 결정값(값1), Negative 결정값(값0)에 따라 결정됨

(1) TN
- 예측값 : Negative 값0, 실제값 : Negative 값 0

(2) FP
- 예측값 : Positive 값1, 실제값 : Negative 값 0

(3) FN
- 예측값 : Negative 값0, 실제값 : Positive 값 1

(4) TP
- 예측값 : Positive 값1, 실제값 : Positive 값 1

오차행렬 API
- confusion_matrix()

예측성능 지표
- MyFakeClassifier

예측 결과
- fakepred

실제 결과
- y_test

In [3]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self, x, y):
        pass
    
    # 입력값으로 들어오는 x 데이터 세트의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, x):
        return np.zeros( (len(x), 1), dtype=bool )
    
# 사이킷런의 내장 데이터 세트인 load_digits()를 이용해 MNIST 데이터 로딩
digits = load_digits()

# digits 번호가 7번이면 True이고 이를 astype(int)로 1로 변환, 7번이 아니면 Falsedlrh 0dmfh qusghks
y = (digits.target == 7).astype(int)
x_train, x_test, y_train, y_test = train_test_split( digits.data, y, random_state=11 )

# 불균형 데이터 생성 = y_test
# 불균형한 레이블 데이터 분포도 확인
# print('레이블 테스트 세트 크기 :', y_test.shape)
# print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

# MyFakeClassifier예측 진행
# Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(x_train, y_train)
fakepred = fakeclf.predict(x_test)
# print('모든 예측을 0으로 하여도 정확도는:{:.3f}'.format(accuracy_score(y_test, fakepred)))


from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, fakepred)

0    405
1     45
Name: count, dtype: int64


array([[405,   0],
       [ 45,   0]])

오차행렬 == ndarray 형태
이진 분류의 TN, FP, FN, TP는 array에서 가져올 수 있음

TN : array[0,0] , 45개

FP : array[0,1], 0개

FN : array[1,0], 45개

TP : array[1,1], 0개

[예제] target==7에 따라 클래스 값 True / False 이진 분류로 변경한 데이터 세트 사용
- 무조건 Negative로 예측
- 데이터 세트의 클래스값 분포도 : 0이 405건, 1이 45건


** TN, FP, FN, TP
- Classfier 성능 판단 기반 정보 제공
- 성능 측정 주요 지표인 정확도(Accuracy), 정밀도(Precision), 재현율(Recall) 값을 알 수 있음

** 정확도 : 예측값과 실제값이 얼마나 동일한가에 대한 비율로 결정됨
=> 오차행렬 True에 해당하는 값인 TN, TP에 의해 결정

** 수식
정확도 = 예측 결과와 실제 값이 동일한 건수 / 전체 데이터수
     = (TN + TP) / (TN + FP + FN + TP)

주로
Positive : 1값 부여
Negative : 0값 부여

<h3>4.3 정밀도와 재현율</h3>
- 정밀도와 재현율은 Positive 데이터 세트의 예측 성능에 초점을 맞춘 평가 지표

** 공식<br>
정밀도 = TP / (FP + TP)<br>
재현율 = TP / (FN + TP)<br>

** 정밀도<br>
- 예측을 Positive로 한 대상중에 예측과 실제 값이 Positive로 일치하는 데이터의 비율
- 분모 (FP + TP) : 예측을 Positive로 한 모든 데이터 건수
- 분자 TP : 예측값과 실제값이 Positive로 일치하는 데이터 건수

** 양성 예측도
- Positive 예측 성능을 더욱 정밀하게 측정하기 위한 평가 지표

** 재현율
- 실제 값이 Positive인 대상 중에 예측과 실제 값이 Positive로 일치한 데이터의 비율
- 분모 (FN + TP) : 실제 값이 Positive인 모든 데이터 건수
- 분자 TP : 예측과 실제 값이 Positive로 일치한 데이터 건수<br>
=> 민감도(Sensitivity) 또는 TPR(True Positive Rate) 라고도 불림


** get_clf_eval()
- confusion matrix, accutacy, precision, recall 의 평가 한번에 호출 가능

In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:4f}, 재현율: {2:4f}'.format(accuracy, precision, recall))
    
    